In [2]:
import os
import sys
import pickle

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.utils import shuffle

import boto3

In [3]:
module_path = os.path.abspath(os.path.join('../../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

using kernel ml.c5.9xlarge

In [4]:
from global_config import ROOT_DIR, emotion_id_to_emotion_abr, conf_cmap
from src.analysis.supervised_learning.evaluation.confusion_matrix import ConfusionMatrixCreator

from src.preprocessing.dataset_creation.aggregation import get_aggregate_measures

In [14]:
def evaluate_scores(X, y, svc, scoring_method):
    
    # get scores
    scores = cross_validate(X=X, y=y,
                            estimator           = svc,
                            scoring             = [scoring_method],
                            verbose             = 1,
                            n_jobs              = -1,
                            return_train_score  = True                        
                           )
    
    print('printing {} measures'.format(scoring_method))
    print('avg (train):', np.mean(scores['train_{}'.format(scoring_method)]))
    print('std (train):', np.std(scores['train_{}'.format(scoring_method)]))
    print('avg (validation):', np.mean(scores['test_{}'.format(scoring_method)]))
    print('std (validation):', np.std(scores['test_{}'.format(scoring_method)]))

In [5]:
from s3fs.core import S3FileSystem
s3 = S3FileSystem()
from sagemaker import get_execution_role

role = get_execution_role()
bucket='files-and-examples-01'
file = 'datasets/su_dataset/sliced_au_gaze_pose.pickle'

path = s3.open('s3://{}/{}'.format(bucket, file))

In [6]:
file = pickle.load(path)

In [7]:
x = file['x']
y = file['y']

au = x['au']
gaze = x['gaze']
pose = x['pose']

# Only means

In [8]:
au_agg = get_aggregate_measures(au,
                               means=True,
                               variance=False,
                               deltas=False,
                               peaks=False)

In [9]:
au_agg.shape

(2640, 85)

In [10]:
y.shape

(2640,)

In [20]:
c_values = [0.1, 1, 5, 10, 20, 50, 80, 100]
gamma = [1, 0.1, 0.01, 0.001, 0.0001]
kernel = ['rbf', 'linear', 'poly', 'sigmoid']

parameters = {'class_weight': ['balanced'],
              'C': c_values,
              'gamma': gamma,
              'kernel': kernel,
              }

svc = SVC()
clf = GridSearchCV(estimator=svc,
                   param_grid=parameters,
                   scoring='accuracy',
                   verbose=5,
                   n_jobs=-1,
                   )

In [21]:
clf.fit(au_agg, y)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 378 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 576 tasks      | elapsed:   38.1s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  1.5min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 5, 10, 20, 50, 80, 100],
                         'class_weight': ['balanced'],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [22]:
clf.best_params_

{'C': 50, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'rbf'}

In [23]:
svc = SVC(**clf.best_params_)

In [25]:
evaluate_scores(au_agg, y, svc, scoring_method="accuracy")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.


printing accuracy measures
avg (train): 0.30359848484848484
std (train): 0.006205361714168902
avg (validation): 0.12727272727272726
std (validation): 0.018533530034431915


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.5s finished


# Adding Variance

In [26]:
au_agg = get_aggregate_measures(au,
                               means=True,
                               variance=True,
                               deltas=False,
                               peaks=False)

In [27]:
clf.fit(au_agg, y)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 378 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done 576 tasks      | elapsed:   38.8s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  1.1min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 5, 10, 20, 50, 80, 100],
                         'class_weight': ['balanced'],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [28]:
clf.best_params_

{'C': 20, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'rbf'}

In [29]:
svc = SVC(**clf.best_params_)

In [30]:
evaluate_scores(au_agg, y, svc, scoring_method="accuracy")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.


printing accuracy measures
avg (train): 0.3135416666666667
std (train): 0.0062843409178170795
avg (validation): 0.12045454545454545
std (validation): 0.014050937114387428


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.8s finished


# Adding deltas

In [31]:
au_agg = get_aggregate_measures(au,
                               means=True,
                               variance=False,
                               deltas=True,
                               peaks=False)

In [32]:
clf.fit(au_agg, y)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done 378 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 576 tasks      | elapsed:   41.1s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  1.2min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 5, 10, 20, 50, 80, 100],
                         'class_weight': ['balanced'],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [33]:
clf.best_params_

{'C': 100, 'class_weight': 'balanced', 'gamma': 0.0001, 'kernel': 'rbf'}

In [34]:
svc = SVC(**clf.best_params_)

In [35]:
evaluate_scores(au_agg, y, svc, scoring_method="accuracy")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.


printing accuracy measures
avg (train): 0.21467803030303031
std (train): 0.005436628823639137
avg (validation): 0.12727272727272726
std (validation): 0.021507681161482408


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.0s finished


# Adding peaks

In [36]:
au_agg = get_aggregate_measures(au,
                               means=True,
                               variance=False,
                               deltas=False,
                               peaks=True)

In [37]:
au_agg.shape

(2640, 102)

In [38]:
clf.best_params_

{'C': 100, 'class_weight': 'balanced', 'gamma': 0.0001, 'kernel': 'rbf'}

In [39]:
svc = SVC(**clf.best_params_)
evaluate_scores(au_agg, y, svc, scoring_method="accuracy")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.


printing accuracy measures
avg (train): 0.4
std (train): 0.012187614047875895
avg (validation): 0.09848484848484848
std (validation): 0.01233243984553008


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.0s finished


# Running on all aggregates 

In [40]:
au_agg = get_aggregate_measures(au)

In [41]:
au_agg.shape

(2640, 153)

In [42]:
clf.fit(au_agg, y)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done 378 tasks      | elapsed:   44.2s
[Parallel(n_jobs=-1)]: Done 576 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  1.9min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 5, 10, 20, 50, 80, 100],
                         'class_weight': ['balanced'],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [43]:
clf.best_params_

{'C': 50, 'class_weight': 'balanced', 'gamma': 0.0001, 'kernel': 'rbf'}

In [44]:
svc = SVC(**clf.best_params_)
evaluate_scores(au_agg, y, svc, scoring_method="accuracy")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.


printing accuracy measures
avg (train): 0.35331439393939396
std (train): 0.006383446797860897
avg (validation): 0.10606060606060605
std (validation): 0.013392173886108855


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.7s finished


# Removing peaks

In [45]:
au_agg = get_aggregate_measures(au,
                               means=True,
                               variance=True,
                               deltas=True,
                               peaks=False)

In [46]:
au_agg.shape

(2640, 136)

In [47]:
clf.fit(au_agg, y)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 378 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 576 tasks      | elapsed:   41.7s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  1.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 5, 10, 20, 50, 80, 100],
                         'class_weight': ['balanced'],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'linear', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [48]:
clf.best_params_

{'C': 20, 'class_weight': 'balanced', 'gamma': 0.001, 'kernel': 'rbf'}

In [49]:
svc = SVC(**clf.best_params_)
evaluate_scores(au_agg, y, svc, scoring_method="accuracy")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.


printing accuracy measures
avg (train): 0.31609848484848485
std (train): 0.007405763180309532
avg (validation): 0.11969696969696968
std (validation): 0.014344012294516277


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.3s finished
